# Introduction

Afin d'atteindre son objectif de ville neutre en émissions carbonnées en 2050, la ville de Seattle s'intéresse aux émissions des bâtiments non résidentiels.

Les agents municipaux ont donc effectué des relevés coûteux et minutieux en 2015 et 2016. À partir de ces relevés, je vais chercher à prédire les émissions de $CO_{2}$ et la consommation totale d'énergie des bâtiments pour lesquels les mesures n'ont pas encore été faites.

La prédiction se basera sur les données déclaratives du permis d'exploitation commerciale (taille et usage des bâtiments, mention de travaux récents, date de construction, etc.)

De plus, l'**[ENERGY STAR Score](https://www.energystar.gov/buildings/benchmark/analyze_benchmarking_results)** étant fastidieux à calculer avec l'approche actuellement utilisée par les équipes, son intérêt pour la prédiction des émissions sera alors évalué.

Ce notebook a pour objectif de présenter l'analyse exploratoire des **[données](https://www.kaggle.com/city-of-seattle/sea-building-energy-benchmarking#2015-building-energy-benchmarking.csv)**.

# Import des librairies et chargement des données

In [1]:
import os

# grammaire abstraite de l'arbre syntaxique de Python
import ast

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

Import des données de 2015

In [54]:
dossier_data = "data/"
nom_fichier = "2015-building-energy-benchmarking.csv"
batiments_2015_data = pd.read_csv(dossier_data+nom_fichier)

In [3]:
batiments_2015_data.head()

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,TaxParcelIdentificationNumber,Location,CouncilDistrictCode,Neighborhood,YearBuilt,...,GHGEmissionsIntensity(kgCO2e/ft2),DefaultData,Comment,ComplianceStatus,Outlier,2010 Census Tracts,Seattle Police Department Micro Community Policing Plan Areas,City Council Districts,SPD Beats,Zip Codes
0,1,2015,NonResidential,Hotel,MAYFLOWER PARK HOTEL,659000030,"{'latitude': '47.61219025', 'longitude': '-122...",7,DOWNTOWN,1927,...,2.64,No,NaN,Compliant,NaN,NaN,14.0,NaN,31.0,18081
1,2,2015,NonResidential,Hotel,PARAMOUNT HOTEL,659000220,"{'latitude': '47.61310583', 'longitude': '-122...",7,DOWNTOWN,1996,...,2.38,No,NaN,Compliant,NaN,NaN,14.0,NaN,31.0,18081
2,3,2015,NonResidential,Hotel,WESTIN HOTEL,659000475,"{'latitude': '47.61334897', 'longitude': '-122...",7,DOWNTOWN,1969,...,1.92,Yes,NaN,Compliant,NaN,NaN,56.0,NaN,31.0,18081
3,5,2015,NonResidential,Hotel,HOTEL MAX,659000640,"{'latitude': '47.61421585', 'longitude': '-122...",7,DOWNTOWN,1926,...,31.38,No,NaN,Compliant,High Outlier,NaN,56.0,NaN,31.0,18081
4,8,2015,NonResidential,Hotel,WARWICK SEATTLE HOTEL,659000970,"{'latitude': '47.6137544', 'longitude': '-122....",7,DOWNTOWN,1980,...,4.02,No,NaN,Compliant,NaN,NaN,56.0,NaN,31.0,19576


In [4]:
batiments_2015_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3340 entries, 0 to 3339
Data columns (total 47 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   OSEBuildingID                                                  3340 non-null   int64  
 1   DataYear                                                       3340 non-null   int64  
 2   BuildingType                                                   3340 non-null   object 
 3   PrimaryPropertyType                                            3340 non-null   object 
 4   PropertyName                                                   3340 non-null   object 
 5   TaxParcelIdentificationNumber                                  3338 non-null   object 
 6   Location                                                       3340 non-null   object 
 7   CouncilDistrictCode                                         

Import des données de 2016

In [55]:
dossier_data = "data/"
nom_fichier = "2016-building-energy-benchmarking.csv"
batiments_2016_data = pd.read_csv(dossier_data+nom_fichier)

In [6]:
batiments_2016_data.head()

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,...,Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
0,1,2016,NonResidential,Hotel,Mayflower park hotel,405 Olive way,Seattle,WA,98101.0,0659000030,...,1.156514e+06,3946027.0,12764.52930,1276453.0,False,NaN,Compliant,NaN,249.98,2.83
1,2,2016,NonResidential,Hotel,Paramount Hotel,724 Pine street,Seattle,WA,98101.0,0659000220,...,9.504252e+05,3242851.0,51450.81641,5145082.0,False,NaN,Compliant,NaN,295.86,2.86
2,3,2016,NonResidential,Hotel,5673-The Westin Seattle,1900 5th Avenue,Seattle,WA,98101.0,0659000475,...,1.451544e+07,49526664.0,14938.00000,1493800.0,False,NaN,Compliant,NaN,2089.28,2.19
3,5,2016,NonResidential,Hotel,HOTEL MAX,620 STEWART ST,Seattle,WA,98101.0,0659000640,...,8.115253e+05,2768924.0,18112.13086,1811213.0,False,NaN,Compliant,NaN,286.43,4.67
4,8,2016,NonResidential,Hotel,WARWICK SEATTLE HOTEL (ID8),401 LENORA ST,Seattle,WA,98121.0,0659000970,...,1.573449e+06,5368607.0,88039.98438,8803998.0,False,NaN,Compliant,NaN,505.01,2.88


In [19]:
batiments_2016_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3376 entries, 0 to 3375
Data columns (total 46 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   OSEBuildingID                    3376 non-null   int64  
 1   DataYear                         3376 non-null   int64  
 2   BuildingType                     3376 non-null   object 
 3   PrimaryPropertyType              3376 non-null   object 
 4   PropertyName                     3376 non-null   object 
 5   Address                          3376 non-null   object 
 6   City                             3376 non-null   object 
 7   State                            3376 non-null   object 
 8   ZipCode                          3360 non-null   float64
 9   TaxParcelIdentificationNumber    3376 non-null   object 
 10  CouncilDistrictCode              3376 non-null   int64  
 11  Neighborhood                     3376 non-null   object 
 12  Latitude            

# Choix des variables

Pour rappel, je cherche à prédire les émissions de $CO_{2}$ (plus précisément les GES) et la consommation totale d'énergie des bâtiments non résidentiels et dont les mesures n'ont pas été effectuées. 

Les variables à prédire sont les suivantes :
- *GHGEmissions (2015)* et *TotalGHGEmissions (2016)* pour les émissions de GES
- *SiteEnergyUse(kBtu)* et *SiteEnergyUseWN(kBtu)* pour la consommation totale d'énergie, le choix devrait se porter sur la première variable car il s'agit de la consommation réelle

Note : *SiteEnergyUseWN(kBtu)* correspond à l'énergie annuelle consommée par le bâtiment ajustée à ce que le bâtiment aurait consommé pendant des conditions météorologiques moyennes de 30 ans. Je ne comprends pas tout à fait ce que cela signifie.

D'après cette **[source](https://www.statistiques.developpement-durable.gouv.fr/sites/default/files/2019-05/datalab-46-chiffres-cles-du-climat-edition-2019-novembre2018.pdf)**, l'énergie consommée par les bâtiments résidentiels est destinée aux usages suivants : chauffage, eau chaude sanitaire et la cuisson. Je vais donc me référer à ces usages pour les bâtiments non résidentiels en prenant en compte en plus les usages liés à l'activité réalisée dans le bâtiment. Par exemple, un café va utiliser de l'électricité pour faire du café ou d'autres boissons, de ce fait, le bâtiment consomme de l'électricité suite à l'activité de l'entreprise.

Les facteurs qui influencent la consommation d'énergie et les émissions de GES des bâtiments non-résidentiels sont les suivants : la taille du bâtiment, la façon dont le bâtiment et l'eau courante sont chauffés (gaz, électricité, fioul, etc.), l'isolation du bâtiment, le type de bâtiment (batîment individuel ou collectif), l'année de construction du bâtiment et l'activité du bâtiment (une boulangerie va consommer de l'électricité pour produire du pain). 

La localisation du bâtiment peut également jouer un rôle dans la consommation d'énergie. En effet, un quartier exposé à un vent froid régulièrement dans l'année verra ses bâtiments consommer plus d'énergie liée au chauffage.

Par conséquent, les variables conservées sont :
- *OSEBuildingID* : ID de la propriété
- *DataYear* : année du relevé pour différencier les données
- *BuildingType* : type de bâtiment selon la classification de Seattle
- *PrimaryPropertyType* : utilisation principale de la propriété (bureaux, université, etc.)
- *Location* : adresse complète de la propriété (uniquement pour 2015)
- *Address* : adresse postale (voie) de la propriété (uniquement pour 2016)
- *City* : ville où est située la propriété (uniquement pour 2016)
- *State* : état où est située la propriété (uniquement pour 2016)
- *ZipCode* : code postal de la propriété (uniquement pour 2016)
- *Neighborhood* : quartier où est située la propriété
- *YearBuilt* : année de construction ou de dernière rénovation complète de la propriété
- *NumberofBuildings* : nombre de bâtiments inclus dans la propriété
- *NumberofFloors* : nombre d'étages
- *PropertyGFABuilding(s)* : surface totale des bâtiments (en $ft^{2}$)
- *ENERGYSTARScore* : évalue la performance énergétique des bâtiments de la propriété, il permet de comparer avec l'ensemble des bâtiments sur le territoire national US. Un score de 50 représente la médiane, un score plus élevé signifie qu'un bâtiment a de meilleures performances énergétiques (moins de consommation) et un score plus faible représente le contraire. Voir **[lien](https://www.energystar.gov/buildings/benchmark/analyze_benchmarking_results)** pour plus d'informations.
- *SiteEnergyUse(kBtu)* : énergie totale consommée par la propriété sur l'année reportée, en prenant en compte toutes les sources d'énergie
- *SteamUse(kBtu)* : énergie consommée par la propriété sur l'année reportée avec de la vapeur
- *Electricity(kBtu)* : énergie consommée par la propriété sur l'année reportée avec de l'électricité
- *NaturalGas(kBtu)* : énergie consommée par la propriété sur l'année reportée avec du gaz naturel
- *OtherFuelUse(kBtu)* : énergie consommée par la propriété sur l'année reportée avec d'autres combustibles (2015 seulement)
- *ComplianceStatus* : statut de la propriété si elle rencontre toutes les conformités du benchmarking réalisé l'année du relevé
- *Outlier* : définit si une propriété est un outlier (Y/N)
- *GHGEmissions(MetricTonsCO2e)* : émissions totales de GES relatives à la consommation énergétique de la propriété (en $t$ $CO_{2}$ $eq$, 2015 uniquement) 
- *TotalGHGEmissions* : émissions totales de GES relatives à la consommation énergétique de la propriété (en $t$ $CO_{2}$ $eq$, 2016 uniquement)

On observe les relations suivantes :
- *SiteEnergyUse(kBtu)* est la somme des variables *SteamUse(kBtu)*, *Electricity(kBtu)*, *NaturalGas(kBtu)* et *OtherFuelUse(kBtu)*
- *GHGEmissions(MetricTonsCO2e)* et *TotalGHGEmissions* sont le résultat d'une combinaison linéaire des mêmes variables citées ci-avant

Notes : 
- le choix de retenir *SourceEUI* a été envisagé car l'*ENERGYSTARScore* utilise cette variable dans ses calculs. Cependant, étant donné les relations décrites ci-dessus, elle ne sera pas retenue.
- l'*ENERGYSTARScore* est calculé grâce à la taille du bâtiment, sa localisation, son nombre d'occupants, son nombre de PC et les données énergétiques relevées.

Attention à bien utiliser les variables relatives aux données déclaratives du permis d'exploitation commerciale et pas les autres.


In [56]:
variables_conservees = ['OSEBuildingID', 'DataYear', 'BuildingType', 'PrimaryPropertyType', 'Neighborhood', 
                        'YearBuilt', 'NumberofBuildings', 'NumberofFloors', 'PropertyGFABuilding(s)', 
                        'ENERGYSTARScore', 'SiteEnergyUse(kBtu)', 'SteamUse(kBtu)', 'Electricity(kBtu)', 
                        'NaturalGas(kBtu)', 'ComplianceStatus', 'Outlier']

variables_specifiques_2015 = ['Location', 'OtherFuelUse(kBtu)', 'GHGEmissions(MetricTonsCO2e)']
variables_specifiques_2016 = ['Address', 'City', 'State', 'ZipCode', 'TotalGHGEmissions']

variables_conservees_2015 = variables_conservees + variables_specifiques_2015
variables_conservees_2016 = variables_conservees + variables_specifiques_2016

In [57]:
batiments_2015_data = batiments_2015_data[variables_conservees_2015]
batiments_2016_data = batiments_2016_data[variables_conservees_2016]

In [58]:
batiments_2015_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3340 entries, 0 to 3339
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   OSEBuildingID                 3340 non-null   int64  
 1   DataYear                      3340 non-null   int64  
 2   BuildingType                  3340 non-null   object 
 3   PrimaryPropertyType           3340 non-null   object 
 4   Neighborhood                  3340 non-null   object 
 5   YearBuilt                     3340 non-null   int64  
 6   NumberofBuildings             3340 non-null   int64  
 7   NumberofFloors                3332 non-null   float64
 8   PropertyGFABuilding(s)        3340 non-null   int64  
 9   ENERGYSTARScore               2560 non-null   float64
 10  SiteEnergyUse(kBtu)           3330 non-null   float64
 11  SteamUse(kBtu)                3330 non-null   float64
 12  Electricity(kBtu)             3330 non-null   float64
 13  Nat

In [10]:
batiments_2016_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3376 entries, 0 to 3375
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   OSEBuildingID           3376 non-null   int64  
 1   DataYear                3376 non-null   int64  
 2   BuildingType            3376 non-null   object 
 3   PrimaryPropertyType     3376 non-null   object 
 4   Neighborhood            3376 non-null   object 
 5   YearBuilt               3376 non-null   int64  
 6   NumberofBuildings       3368 non-null   float64
 7   NumberofFloors          3376 non-null   int64  
 8   PropertyGFABuilding(s)  3376 non-null   int64  
 9   ENERGYSTARScore         2533 non-null   float64
 10  SiteEnergyUse(kBtu)     3371 non-null   float64
 11  SteamUse(kBtu)          3367 non-null   float64
 12  Electricity(kBtu)       3367 non-null   float64
 13  NaturalGas(kBtu)        3367 non-null   float64
 14  ComplianceStatus        3376 non-null   

# Analyses exploratoires

## Analyses préliminaires

### Y a-t-il les mêmes propriétés dans les deux datasets ?

In [59]:
liste_ID_proprietes_communes = batiments_2015_data.merge(batiments_2016_data, on='OSEBuildingID')['OSEBuildingID']

In [60]:
len(liste_ID_proprietes_communes)

3284

Il y a 3284 propriétés présentes dans chacun des deux datasets. Je garderai donc les données de 2016 pour ces propriétés, je les supprime donc du dataset de 2015.

In [61]:
batiments_2015_data = batiments_2015_data[~batiments_2015_data.OSEBuildingID.isin(liste_ID_proprietes_communes)]
batiments_2015_data.reset_index(drop=True, inplace=True)

In [62]:
batiments_2015_data.head()

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,Neighborhood,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFABuilding(s),ENERGYSTARScore,SiteEnergyUse(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu),ComplianceStatus,Outlier,Location,OtherFuelUse(kBtu),GHGEmissions(MetricTonsCO2e)
0,61,2015,Multifamily LR (1-4),Low-Rise Multifamily,DOWNTOWN,1900,1,4.0,48383,75.0,2953338.0,915977.0,1448859.0,588563.0,Compliant,NaN,"{'latitude': '47.60973588', 'longitude': '-122...",0.0,112.06
1,266,2015,Multifamily LR (1-4),Low-Rise Multifamily,DELRIDGE,2007,1,4.0,52134,85.0,1212551.0,0.0,1212601.0,0.0,Compliant,NaN,"{'latitude': '47.54423839', 'longitude': '-122...",0.0,8.45
2,325,2015,NonResidential,Retail Store,DOWNTOWN,1989,1,4.0,111077,91.0,9898724.0,0.0,9899135.0,0.0,Compliant,NaN,"{'latitude': '47.61171108', 'longitude': '-122...",0.0,69.01
3,400,2015,NonResidential,Small- and Mid-Sized Office,DOWNTOWN,1906,1,6.0,72450,45.0,6525887.0,0.0,6488262.0,37893.0,Compliant,NaN,"{'latitude': '47.60679898', 'longitude': '-122...",0.0,47.24
4,442,2015,NonResidential,Large Office,LAKE UNION,1947,1,4.0,155934,59.0,16760217.0,0.0,16760914.0,0.0,Compliant,NaN,"{'latitude': '47.6214758', 'longitude': '-122....",0.0,116.84


In [63]:
batiments_2015_data.shape

(56, 19)

Préparation des données de 2015 pour les concaténer avec celles de 2016 :
- création des variables *Address*, *City*, *State* et *ZipCode* avec *Location*
- suppression de *Location* et des variables intermédiaires
- renommage de *GHGEmissions(MetricTonsCO2e)* en *GHGEmissions*

In [64]:
# conversion d'un string en dictionnaire
def convertir_string_en_dict(string):
    return ast.literal_eval(string)

In [65]:
# création des nouvelles variables
nb_lignes = len(batiments_2015_data)
batiments_2015_data['Location_dict'] = [convertir_string_en_dict(batiments_2015_data.Location[i]) for i in range(nb_lignes)]
batiments_2015_data['human_address'] = [convertir_string_en_dict(batiments_2015_data.Location_dict[i]['human_address']) 
                                            for i in range(nb_lignes)]
batiments_2015_data['Address'] = [batiments_2015_data['human_address'][i]['address'] for i in range(nb_lignes)]
batiments_2015_data['City'] = [batiments_2015_data['human_address'][i]['city'] for i in range(nb_lignes)]
batiments_2015_data['State'] = [batiments_2015_data['human_address'][i]['state'] for i in range(nb_lignes)]
batiments_2015_data['ZipCode'] = [batiments_2015_data['human_address'][i]['zip'] for i in range(nb_lignes)]

In [66]:
# suppression des variables intermédiaires
batiments_2015_data.drop(columns=['Location', 'Location_dict', 'human_address'], inplace=True)

In [67]:
# renommage de la variable GHGEmissions(MetricTonsCO2e)
batiments_2015_data.rename(columns = {'GHGEmissions(MetricTonsCO2e)':'GHGEmissions'}, inplace=True)

In [68]:
batiments_2015_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   OSEBuildingID           56 non-null     int64  
 1   DataYear                56 non-null     int64  
 2   BuildingType            56 non-null     object 
 3   PrimaryPropertyType     56 non-null     object 
 4   Neighborhood            56 non-null     object 
 5   YearBuilt               56 non-null     int64  
 6   NumberofBuildings       56 non-null     int64  
 7   NumberofFloors          56 non-null     float64
 8   PropertyGFABuilding(s)  56 non-null     int64  
 9   ENERGYSTARScore         41 non-null     float64
 10  SiteEnergyUse(kBtu)     55 non-null     float64
 11  SteamUse(kBtu)          55 non-null     float64
 12  Electricity(kBtu)       55 non-null     float64
 13  NaturalGas(kBtu)        55 non-null     float64
 14  ComplianceStatus        56 non-null     obje

Préparation des données de 2016 pour les concaténer avec celles de 2015 :
- création de la variable *OtherFuelUse(kBtu)*
- remplissage de cette variable avec des 0
- renommage de TotalGHGEmissions en GHGEmissions

In [69]:
# création de la variable OtherFuelUse(kBtu) et remplissage avec des 0
batiments_2016_data['OtherFuelUse(kBtu)'] = 0

In [70]:
# renommage de la variable TotalGHGEmissions
batiments_2016_data.rename(columns = {'TotalGHGEmissions':'GHGEmissions'}, inplace=True)

In [71]:
batiments_2016_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3376 entries, 0 to 3375
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   OSEBuildingID           3376 non-null   int64  
 1   DataYear                3376 non-null   int64  
 2   BuildingType            3376 non-null   object 
 3   PrimaryPropertyType     3376 non-null   object 
 4   Neighborhood            3376 non-null   object 
 5   YearBuilt               3376 non-null   int64  
 6   NumberofBuildings       3368 non-null   float64
 7   NumberofFloors          3376 non-null   int64  
 8   PropertyGFABuilding(s)  3376 non-null   int64  
 9   ENERGYSTARScore         2533 non-null   float64
 10  SiteEnergyUse(kBtu)     3371 non-null   float64
 11  SteamUse(kBtu)          3367 non-null   float64
 12  Electricity(kBtu)       3367 non-null   float64
 13  NaturalGas(kBtu)        3367 non-null   float64
 14  ComplianceStatus        3376 non-null   

Concaténation des deux datasets.

In [72]:
batiments_data = pd.concat([batiments_2015_data, batiments_2016_data])

In [73]:
batiments_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3432 entries, 0 to 3375
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   OSEBuildingID           3432 non-null   int64  
 1   DataYear                3432 non-null   int64  
 2   BuildingType            3432 non-null   object 
 3   PrimaryPropertyType     3432 non-null   object 
 4   Neighborhood            3432 non-null   object 
 5   YearBuilt               3432 non-null   int64  
 6   NumberofBuildings       3424 non-null   float64
 7   NumberofFloors          3432 non-null   float64
 8   PropertyGFABuilding(s)  3432 non-null   int64  
 9   ENERGYSTARScore         2574 non-null   float64
 10  SiteEnergyUse(kBtu)     3426 non-null   float64
 11  SteamUse(kBtu)          3422 non-null   float64
 12  Electricity(kBtu)       3422 non-null   float64
 13  NaturalGas(kBtu)        3422 non-null   float64
 14  ComplianceStatus        3432 non-null   

In [74]:
batiments_data

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,Neighborhood,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFABuilding(s),ENERGYSTARScore,...,Electricity(kBtu),NaturalGas(kBtu),ComplianceStatus,Outlier,OtherFuelUse(kBtu),GHGEmissions,Address,City,State,ZipCode
0,61,2015,Multifamily LR (1-4),Low-Rise Multifamily,DOWNTOWN,1900,1.0,4.0,48383,75.0,...,1.448859e+06,5.885630e+05,Compliant,NaN,0.0,112.06,107 PINE ST,SEATTLE,WA,98101
1,266,2015,Multifamily LR (1-4),Low-Rise Multifamily,DELRIDGE,2007,1.0,4.0,52134,85.0,...,1.212601e+06,0.000000e+00,Compliant,NaN,0.0,8.45,6500 35TH AVE SW,SEATTLE,WA,98126
2,325,2015,NonResidential,Retail Store,DOWNTOWN,1989,1.0,4.0,111077,91.0,...,9.899135e+06,0.000000e+00,Compliant,NaN,0.0,69.01,400 PINE ST,SEATTLE,WA,98101
3,400,2015,NonResidential,Small- and Mid-Sized Office,DOWNTOWN,1906,1.0,6.0,72450,45.0,...,6.488262e+06,3.789300e+04,Compliant,NaN,0.0,47.24,1201 2ND AVE,SEATTLE,WA,98101
4,442,2015,NonResidential,Large Office,LAKE UNION,1947,1.0,4.0,155934,59.0,...,1.676091e+07,0.000000e+00,Compliant,NaN,0.0,116.84,333 DEXTER AVE N,SEATTLE,WA,98109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3371,50222,2016,Nonresidential COS,Office,GREATER DUWAMISH,1990,1.0,1.0,12294,46.0,...,5.242709e+05,3.254750e+05,Error - Correct Default Data,NaN,0.0,20.94,1600 S Dakota St,Seattle,WA,NaN
3372,50223,2016,Nonresidential COS,Other,DOWNTOWN,2004,1.0,1.0,16000,NaN,...,3.965461e+05,5.537300e+05,Compliant,NaN,0.0,32.17,719 8th Ave S,Seattle,WA,NaN
3373,50224,2016,Nonresidential COS,Other,MAGNOLIA / QUEEN ANNE,1974,1.0,1.0,13157,NaN,...,1.792159e+06,3.973739e+06,Compliant,NaN,0.0,223.54,1920 1st Ave W,Seattle,WA,NaN
3374,50225,2016,Nonresidential COS,Mixed Use Property,GREATER DUWAMISH,1989,1.0,1.0,14101,NaN,...,3.488702e+05,3.706010e+05,Compliant,NaN,0.0,22.11,8319 8th Ave S,Seattle,WA,NaN
